Create a query function that sums over the database and adds just the right amount of noise such that it satisfies the epsilon constraint for laplacian noise.

In [ ]:
!pip install syft==0.2.9 >/dev/null

ERROR: tensorflow 2.4.1 has requirement numpy~=1.19.2, but you'll have numpy 1.18.5 which is incompatible.
ERROR: google-colab 1.0.0 has requirement notebook~=5.3.0; python_version >= "3.0", but you'll have notebook 5.7.8 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement tornado~=5.1.0; python_version >= "3.0", but you'll have tornado 4.5.3 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: bokeh 2.1.1 has requirement tornado>=5.1, but you'll have tornado 4.5.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


In [ ]:
import numpy as np
import torch

class CalculateSensitivity:
    "This is a class that contains reusable methods for initializing parallel dbs \
with number of entries. Use classObject.create_db_and_parallels(num_entries) to \
get db and parallel db list. Use classObject.sensitivity(query, num_entries, verbose) \
to find the sensitivity of the query function"

    def get_parallel_db(self, db, remove_index):
        return torch.cat((db[0:remove_index], db[remove_index+1:]))

    def get_parallel_dbs(self, db):
        parallel_dbs = list()
        for i in range(len(db)):
            pdb = self.get_parallel_db(db, i)
            parallel_dbs.append(pdb)
        return parallel_dbs

    def create_db_and_parallels(self, num_entries):
        db = torch.rand(num_entries).gt(0.5).to(torch.uint8)
        pdbs = self.get_parallel_dbs(db)
        return db, pdbs

    def sensitivity(self, query, num_entries, verbose):

        db, pdbs = self.create_db_and_parallels(num_entries)

        if verbose:
            print('Centralized data is: ', db)
            print('Parallel data are: ', pdbs)

        centralized_result = query(db)
        parallel_results = [query(temp_db) for temp_db in pdbs]

        if verbose:
            print('Centralized result: ', centralized_result)
            print('Parallel results: ', parallel_results)

        max_distance = 0
        for parallel_result in parallel_results:
            if max_distance < torch.abs(centralized_result - parallel_result): 
                max_distance = torch.abs(centralized_result - parallel_result)
        L1_sensitivity = max_distance
        return L1_sensitivity

helper = CalculateSensitivity()

In [ ]:

epsilon = 0.5

db, pdbs = helper.create_db_and_parallels(num_entries=100)

sum(db)

tensor(52, dtype=torch.uint8)

In [ ]:
def sum_query(db):
    return db.sum()

def laplacian_mechanism(db, query, sensitivity):
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    return query(db) + noise # Global DP

# Sum query's sensitivity is 1 as for a binary array, the maximum change that can be made by removing an element is by 1
laplacian_mechanism(db, sum_query, 1)

tensor([52.0594], dtype=torch.float64)

In [ ]:
# True query return
sum_query(db)

tensor(52)

That's pretty close

Doing the same for mean query

In [ ]:
def mean_query(db):
    return torch.mean(db.float())

# The sensitivity of this query is 1/100. That is the maximum amount by which the sum can change divided
# by the total number of numbers in the db

In [ ]:
# laplacian mechanism on mean query
laplacian_mechanism(db, mean_query, 1/100)

tensor([0.5079], dtype=torch.float64)

In [ ]:
mean_query(db)

tensor(0.5200)

Lower values of epsilon like 0.0001 etc will have a negative impact on the query's output as it will add too much noise. ie beta for laplace = sensitivity / epsilon